# Sygus completions

In [1]:
import os
import sys
from pathlib import Path

CURRENT_DIRECTORY = Path(os.getcwd())
ROOT_DIRECTORY = (CURRENT_DIRECTORY / "..").absolute().resolve()

print(f"Current directory: {CURRENT_DIRECTORY}")
print(f"Root directory: {ROOT_DIRECTORY}")

sys.path.append(str(ROOT_DIRECTORY))

Current directory: /home/ubuntu/arga-arc/sygus
Root directory: /home/ubuntu/arga-arc


In [43]:
import typing as t
from pprint import pprint
from dataclasses import dataclass
import sexpdata as sexp
from sexpdata import Symbol
from openai import OpenAI
from config import CONFIG
from datetime import datetime
import json

OPENAI = OpenAI(organization=CONFIG.OPENAI_ORGANIZATION, api_key=CONFIG.OPENAI_SECRET_KEY)

## Generating Completions

In [3]:
BENCHMARKS_DIRECTORY = ROOT_DIRECTORY / "sygus/Probe/src/test/benchmarks"
CIRCUIT_DIRECTORY = BENCHMARKS_DIRECTORY / "circuit/test"
HACKERS_DELIGHT_DIRECTORY = BENCHMARKS_DIRECTORY / "hackers-delight"
LARGER_STRING_GRAMMAR_DIRECTORY = BENCHMARKS_DIRECTORY / "larger-grammar"

In [4]:
# for each file in the CIRCUIT_DIRECTORY, print the filename
for file in CIRCUIT_DIRECTORY.iterdir():
    print(file)

/home/ubuntu/arga-arc/sygus/Probe/src/test/benchmarks/circuit/test/CrCy_10-sbox2-D7-sIn1.sl
/home/ubuntu/arga-arc/sygus/Probe/src/test/benchmarks/circuit/test/CrCy_6-P10-D7-sIn.sl
/home/ubuntu/arga-arc/sygus/Probe/src/test/benchmarks/circuit/test/CrCy_6-P10-D7-sIn3.sl
/home/ubuntu/arga-arc/sygus/Probe/src/test/benchmarks/circuit/test/CrCy_10-sbox2-D5-sIn104.sl
/home/ubuntu/arga-arc/sygus/Probe/src/test/benchmarks/circuit/test/CrCy_9-pprmAll-D7-sIn1.sl
/home/ubuntu/arga-arc/sygus/Probe/src/test/benchmarks/circuit/test/CrCy_6-P10-D5-sIn.sl
/home/ubuntu/arga-arc/sygus/Probe/src/test/benchmarks/circuit/test/CrCy_10-sbox2-D5-sIn88.sl
/home/ubuntu/arga-arc/sygus/Probe/src/test/benchmarks/circuit/test/CrCy_8-P12-D7-sIn5.sl
/home/ubuntu/arga-arc/sygus/Probe/src/test/benchmarks/circuit/test/CrCy_10-sbox2-D5-sIn89.sl
/home/ubuntu/arga-arc/sygus/Probe/src/test/benchmarks/circuit/test/CrCy_2-P6_2-P6.sl
/home/ubuntu/arga-arc/sygus/Probe/src/test/benchmarks/circuit/test/CrCy_6-P10-D7-sIn5.sl
/home/u

In [5]:
SYSTEM_PROMPT = """You are a coding assistant. Be precise and terse.
You will be given a SyGuS grammar, a natural language specification, and a set of input-output examples.
Your task is to write a program that is correct according to the grammar, specification, and examples."""

In [6]:
TEST_GRAMMAR = """(synth-fun f ((x (BitVec 64))) (BitVec 64)
    ((Start (BitVec 64) ((bvnot Start)
						 (bvxor Start Start)
						 (bvand Start Start)
						 (bvor Start Start)
						 (bvneg Start)
						 (bvadd Start Start)
						 (bvmul Start Start)
						 (bvudiv Start Start)
						 (bvurem Start Start)
						 (bvlshr Start Start)
						 (bvashr Start Start)
						 (bvshl Start Start)
						 (bvsdiv Start Start)
						 (bvsrem Start Start)
						 (bvsub Start Start)
                         x
						 #x0000000000000000
                         #x0000000000000001
                         #x0000000000000002
                         #x000000000000001f
                         #xffffffffffffffff
                         (ite StartBool Start Start)))

                         (StartBool Bool
                         ((= Start Start)
                         ))))"""

TEST_NAT_LANG_SPEC = "; Next higher unsigned number with the same number of 1 bits."

TEST_EXAMPLES = [
    ("#x0000000000000001", "#x0000000000000002"),
    ("#x0000000000000003", "#x0000000000000005"),
]

USER_MESSAGE = f"""[GRAMMAR]
{TEST_GRAMMAR}

[NATURAL LANGUAGE SPECIFICATION]
{TEST_NAT_LANG_SPEC}

[EXAMPLES]
{[f"{x} -> {y}" for x, y in TEST_EXAMPLES]}

[SOLUTION]
(define-fun ans ((x (BitVec 64))) (BitVec 64)"""

In [7]:
client = OpenAI(organization=CONFIG.OPENAI_ORGANIZATION, api_key=CONFIG.OPENAI_SECRET_KEY)

response = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": USER_MESSAGE},
  ],
  n=10,
  temperature=0.5,
)
pprint([choice.message.content for choice in response.choices])

['(bvadd x #x0000000000000001))',
 '(bvadd x #x0000000000000001))',
 '(bvadd x #x0000000000000001))',
 '(bvadd x #x0000000000000001))',
 '(let ((t (bvand x (bvsub x #x0000000000000001))))\n'
 '  (bvlshr (bvadd (bvxor x t) #x0000000000000001) (bvadd (bvnot (bvurem '
 '(bvlshr (bvnot t) #x0000000000000001) #x000000000000001f)) '
 '#x0000000000000001))))',
 '(bvadd x #x0000000000000001))',
 '(bvadd x #x0000000000000001))',
 '(bvadd x #x0000000000000001))',
 '(let (\n'
 '    (a (bvand x (bvsub x #x0000000000000001)))\n'
 '    (b (bvsub x a))\n'
 '    (c (bvsub (bvand b #xffffffffffffffff) #x0000000000000001))\n'
 '    (d (bvurem (bvsub b #x0000000000000001) c))\n'
 '    (e (bvadd d #x0000000000000001))\n'
 '    (f (bvsub (bvshl #x0000000000000001 e) #x0000000000000001))\n'
 '    (g (bvand f (bvnot b)))\n'
 '    (h (bvlshr g e))\n'
 '    (i (bvsub x h))\n'
 '    (j (bvsub (bvshl #x0000000000000001 e) #x0000000000000001))\n'
 '    (k (bvand j (bvnot b)))\n'
 '    (l (bvlshr k e))\n'
 '    )\

### Larger String Grammar

In [8]:
LARGER_STRING_GRAMMAR_PROBLEMS = {}

LARGER_STRING_GRAMMAR_COMMENTS = {}

def is_comment(line: str) -> bool:
    return line.strip().startswith(";")


# for each file in the CIRCUIT_DIRECTORY, print the filename
for file in LARGER_STRING_GRAMMAR_DIRECTORY.iterdir():
    print(file.name)
    with open(file, "r") as f:
        contents = file.read_text()
        LARGER_STRING_GRAMMAR_PROBLEMS[file.name] = sexp.loads(contents)
        LARGER_STRING_GRAMMAR_COMMENTS[file.name] = [
            line for line in contents.split("\n") if is_comment(line)
        ]

exceljet2modified.sl
31753108modified.sl
find-nth-occurrence-of-charactermodified.sl
clean-and-reformat-telephone-numbersmodified.sl
17212077modified.sl
11604909modified.sl
stackoverflow4modified.sl
remove-text-by-positionmodified.sl
44789427modified.sl
stackoverflow11modified.sl
stackoverflow3modified.sl
initials-longmodified.sl
phone-7modified.sl
43606446modified.sl
extract-nth-word-from-text-stringmodified.sl
remove-leading-and-trailing-spaces-from-textmodified.sl
phone-5-long-repeatmodified.sl
count-total-words-in-a-cellmodified.sl
stackoverflow8modified.sl
strip-numeric-characters-from-cellmodified.sl
phone-9modified.sl
initialsmodified.sl
phone-10-long-repeatmodified.sl
39060015modified.sl
phone-7-long-repeatmodified.sl
stackoverflow6modified.sl
strip-non-numeric-charactersmodified.sl
stackoverflow1modified.sl
get-last-wordmodified.sl
phone-5modified.sl
phone-6-long-repeatmodified.sl
get-last-line-in-cellmodified.sl
phone-9-shortmodified.sl
38871714modified.sl
phone-5-shortmodifi

In [9]:
pprint(LARGER_STRING_GRAMMAR_PROBLEMS["exceljet2modified.sl"][1])
pprint(LARGER_STRING_GRAMMAR_COMMENTS["exceljet2modified.sl"])

[Symbol('synth-fun'),
 Symbol('f'),
 [[Symbol('_arg_0'), Symbol('String')]],
 Symbol('String'),
 [[Symbol('Start'), Symbol('String'), [Symbol('ntString')]],
  [Symbol('ntString'),
   Symbol('String'),
   [Symbol('_arg_0'),
    '',
    ' ',
    'BRD',
    'DRS',
    'LDS',
    'Branding',
    'Direct Response',
    'Leads',
    '=',
    '/',
    'in',
    '_',
    '9',
    '.',
    'microsoft',
    'windows',
    'apple',
    'mac',
    '-',
    '1',
    '2',
    '3',
    '4',
    '5',
    '6',
    '7',
    '8',
    '0',
    ',',
    '<',
    '>',
    '/n',
    '%',
    'b',
    'apple',
    'bananas',
    'strawberries',
    'oranges',
    'LLC',
    'Inc',
    'Corporation',
    'Enterprises',
    'Company',
    '(',
    ')',
    '+',
    'name',
    ',',
    [Symbol('str.++'), Symbol('ntString'), Symbol('ntString')],
    [Symbol('str.replace'),
     Symbol('ntString'),
     Symbol('ntString'),
     Symbol('ntString')],
    [Symbol('str.at'), Symbol('ntString'), Symbol('ntInt')],
    

In [35]:
def is_synth_fun(sexp) -> bool:
    ans = isinstance(sexp[0], Symbol) and sexp[0].value() == "synth-fun"
    return ans

def get_synth_fun(sexps: t.List[t.Any]) -> t.Optional[t.Any]:
    for sexp in sexps:
        if is_synth_fun(sexp):
            return sexp
    return None

def get_signature(synth_fun: t.Any) -> t.Tuple[str, t.List[t.Any], t.Any]:
    name = synth_fun[1].value()
    args = [(arg[0].value(), arg[1].value()) for arg in synth_fun[2]]
    ret_type = synth_fun[3].value()
    return (name, args, ret_type)

def is_constraint(sexp) -> bool:
    return isinstance(sexp[0], Symbol) and sexp[0].value() == "constraint"

def get_constraints(sexps: t.List[t.Any]) -> t.List[t.Any]:
    return [sexp for sexp in sexps if is_constraint(sexp)]

def constraint_to_io(sexp, num_args: int) -> t.Tuple[t.List[str], str]:
    assert is_constraint(sexp)
    eq_exp = sexp[1]
    f_exp = eq_exp[1]
    output_exp = eq_exp[2]
    f_args_exp = f_exp[1:]
    assert len(f_args_exp) == num_args
    return ([str(arg) for arg in f_args_exp], output_exp)

@dataclass
class SygusProblem:
    synth_fun: str
    signature: t.Tuple[str, t.List[t.Any], t.Any]
    examples: t.List[t.Tuple[str, str]]
    natural_language_spec: str

    @classmethod
    def from_sexps(cls, sexps: t.List[t.Any], comments: t.List[str]) -> "SygusProblem":
        synth_fun = get_synth_fun(sexps)
        assert synth_fun is not None
        synth_fun_str = sexp.dumps(synth_fun)
        
        signature = get_signature(synth_fun)
        num_args = len(signature[1])

        constraints = get_constraints(sexps)
        examples = [constraint_to_io(constraint, num_args) for constraint in constraints]
        return cls(synth_fun=synth_fun_str, signature=signature, examples=examples, natural_language_spec="\n".join(comments))
    
    @property
    def num_args(self) -> int:
        return len(self.signature[1])
    
    @property
    def function_definition_prefix(self) -> str:
        return f"(define-fun {self.signature[0]} ({' '.join([f'{arg[0]} {arg[1]}' for arg in self.signature[1]])}) {self.signature[2]}"
    
    def completion_to_function_definition(self, completion: str) -> str:
        return f"{self.function_definition_prefix}\n{completion}"
    
    @property
    def user_message(self) -> str:
        EXAMPLES = ""
        for args, output in self.examples:
            EXAMPLES += f"{', '.join(args)} -> {output}\n"
        return f"""[GRAMMAR]
{self.synth_fun}

[NATURAL LANGUAGE SPECIFICATION]
{self.natural_language_spec}

[EXAMPLES]
{EXAMPLES}

[SOLUTION]
{self.function_definition_prefix}"""

In [36]:
LARGER_STRING_GRAMMAR_SYGUS = {}
for filename, sexps in LARGER_STRING_GRAMMAR_PROBLEMS.items():
    pprint(filename)
    LARGER_STRING_GRAMMAR_SYGUS[filename] = SygusProblem.from_sexps(sexps, LARGER_STRING_GRAMMAR_COMMENTS[filename])

'exceljet2modified.sl'
'31753108modified.sl'
'find-nth-occurrence-of-charactermodified.sl'
'clean-and-reformat-telephone-numbersmodified.sl'
'17212077modified.sl'
'11604909modified.sl'
'stackoverflow4modified.sl'
'remove-text-by-positionmodified.sl'
'44789427modified.sl'
'stackoverflow11modified.sl'
'stackoverflow3modified.sl'
'initials-longmodified.sl'
'phone-7modified.sl'
'43606446modified.sl'
'extract-nth-word-from-text-stringmodified.sl'
'remove-leading-and-trailing-spaces-from-textmodified.sl'
'phone-5-long-repeatmodified.sl'
'count-total-words-in-a-cellmodified.sl'
'stackoverflow8modified.sl'
'strip-numeric-characters-from-cellmodified.sl'
'phone-9modified.sl'
'initialsmodified.sl'
'phone-10-long-repeatmodified.sl'
'39060015modified.sl'
'phone-7-long-repeatmodified.sl'
'stackoverflow6modified.sl'
'strip-non-numeric-charactersmodified.sl'
'stackoverflow1modified.sl'
'get-last-wordmodified.sl'
'phone-5modified.sl'
'phone-6-long-repeatmodified.sl'
'get-last-line-in-cellmodified.sl'


In [37]:
pprint(LARGER_STRING_GRAMMAR_SYGUS["exceljet2modified.sl"].user_message)

('[GRAMMAR]\n'
 '(synth-fun f ((_arg_0 String)) String ((Start String (ntString)) (ntString '
 'String (_arg_0 "" " " "BRD" "DRS" "LDS" "Branding" "Direct Response" "Leads" '
 '"=" "/" "in" "_" "9" "." "microsoft" "windows" "apple" "mac" "-" "1" "2" "3" '
 '"4" "5" "6" "7" "8" "0" "," "<" ">" "/n" "%" "b" "apple" "bananas" '
 '"strawberries" "oranges" "LLC" "Inc" "Corporation" "Enterprises" "Company" '
 '"(" ")" "+" "name" "," (str.++ ntString ntString) (str.replace ntString '
 'ntString ntString) (str.at ntString ntInt) (int.to.str ntInt) (ite ntBool '
 'ntString ntString) (str.substr ntString ntInt ntInt))) (ntInt Int (-1 1 2 3 '
 '4 5 6 7 8 9 0 1 0 -1 (+ ntInt ntInt) (- ntInt ntInt) (str.len ntString) '
 '(str.to.int ntString) (ite ntBool ntInt ntInt) (str.indexof ntString '
 'ntString ntInt))) (ntBool Bool (true false (= ntInt ntInt) (str.prefixof '
 'ntString ntString) (str.suffixof ntString ntString) (str.contains ntString '
 'ntString)))))\n'
 '\n'
 '[NATURAL LANGUAGE SPECIFICAT

In [38]:
for filename, problem in LARGER_STRING_GRAMMAR_SYGUS.items():
    pprint(filename)
    pprint(problem.signature)
    pprint(problem.examples)

'exceljet2modified.sl'
('f', [('_arg_0', 'String')], 'String')
[(['www.domain.com'], 'com'),
 (['mail.net'], 'net'),
 (['www.amaon.co.uk'], 'uk')]
'31753108modified.sl'
('f', [('_arg_0', 'String')], 'String')
[(['Tire Pressure ABC123873 Monitor'], 'ABC123873'),
 ([' Oil Life ABC849999999021 gauge'], 'ABC849999999021'),
 ([' Air conditioner GHF211 maintenance'], 'GHF211')]
'find-nth-occurrence-of-charactermodified.sl'
('f', [('_arg_0', 'String'), ('_arg_1', 'Int')], 'Int')
[(['replies to _aya, _tasisuke, and _chan', '1'], 12),
 (['replies to _aya, _tasisuke, and _chan', '2'], 18),
 (['replies to _aya, _tasisuke, and _chan', '3'], 33)]
'clean-and-reformat-telephone-numbersmodified.sl'
('f', [('_arg_0', 'String')], 'String')
[(['801-456-8765'], '8014568765'),
 (['<978> 654-0299'], '9786540299'),
 (['978.654.0299'], '9786540299')]
'17212077modified.sl'
('f', [('_arg_0', 'String')], 'String')
[(['01/15/2013'], '01/2013'),
 (['03/07/2011'], '03/2011'),
 (['05/09/2009'], '05/2009')]
'11604909

In [39]:
def sample_gpt_solutions(problem: SygusProblem, n: int = 10) -> t.Tuple[t.List[str], int]:
    start_time = datetime.now()
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": problem.user_message},
        ],
        n=n,
        temperature=0.5,
    )
    end_time = datetime.now()
    time_diff_ms = (end_time - start_time).microseconds / 1000
    return [problem.completion_to_function_definition(choice.message.content) for choice in response.choices], time_diff_ms

In [40]:
sample_gpt_solutions(LARGER_STRING_GRAMMAR_SYGUS["exceljet2modified.sl"])

(['(define-fun f (_arg_0 String) String\n(str.substr _arg_0 (+ 1 (str.indexof _arg_0 "." (- (str.len _arg_0) 1))) (- (str.len _arg_0) (+ 1 (str.indexof _arg_0 "." (- (str.len _arg_0) 1))))))',
  '(define-fun f (_arg_0 String) String\n(str.substr _arg_0 (+ (str.indexof _arg_0 "." (- (str.len _arg_0) 1)) 1) (- (str.len _arg_0) (+ (str.indexof _arg_0 "." (- (str.len _arg_0) 1)) 1))))',
  '(define-fun f (_arg_0 String) String\n(str.substr _arg_0 (+ 1 (str.indexof _arg_0 "." (- (str.len _arg_0) 1))) (- (str.len _arg_0) (+ 1 (str.indexof _arg_0 "." (- (str.len _arg_0) 1))))))',
  '(define-fun f (_arg_0 String) String\n(str.substr _arg_0 (+ (str.indexof _arg_0 "." (- (str.len _arg_0) 1)) 1) (- (str.len _arg_0) (+ (str.indexof _arg_0 "." (- (str.len _arg_0) 1)) 1))))',
  '(define-fun f (_arg_0 String) String\n(str.substr _arg_0 (+ (str.indexof _arg_0 "." (- (str.len _arg_0) 1)) 1) (- (str.len _arg_0) (+ (str.indexof _arg_0 "." (- (str.len _arg_0) 1)) 1))))',
  '(define-fun f (_arg_0 String) St

In [41]:
LARGER_STRING_GRAMMAR_OUTPUT = {}

for filename, problem in LARGER_STRING_GRAMMAR_SYGUS.items():
    print(filename)
    solutions, time_diff_ms = sample_gpt_solutions(problem)
    LARGER_STRING_GRAMMAR_OUTPUT[filename] = {
        "solutions": solutions,
        "time_ms": time_diff_ms
    }

exceljet2modified.sl


31753108modified.sl
find-nth-occurrence-of-charactermodified.sl
clean-and-reformat-telephone-numbersmodified.sl
17212077modified.sl
11604909modified.sl
stackoverflow4modified.sl
remove-text-by-positionmodified.sl
44789427modified.sl
stackoverflow11modified.sl
stackoverflow3modified.sl
initials-longmodified.sl
phone-7modified.sl
43606446modified.sl
extract-nth-word-from-text-stringmodified.sl
remove-leading-and-trailing-spaces-from-textmodified.sl
phone-5-long-repeatmodified.sl
count-total-words-in-a-cellmodified.sl
stackoverflow8modified.sl
strip-numeric-characters-from-cellmodified.sl
phone-9modified.sl
initialsmodified.sl
phone-10-long-repeatmodified.sl
39060015modified.sl
phone-7-long-repeatmodified.sl
stackoverflow6modified.sl
strip-non-numeric-charactersmodified.sl
stackoverflow1modified.sl
get-last-wordmodified.sl
phone-5modified.sl
phone-6-long-repeatmodified.sl
get-last-line-in-cellmodified.sl
phone-9-shortmodified.sl
38871714modified.sl
phone-5-shortmodified.sl
initials_smallm

In [44]:
LARGER_STRING_GRAMMAR_OUTPUT_FILE = ROOT_DIRECTORY / "sygus/larger-string-grammar-completions.json"
LARGER_STRING_GRAMMAR_OUTPUT_FILE.write_text(json.dumps(LARGER_STRING_GRAMMAR_OUTPUT, indent=2))

112525

## Hackers Delight

In [ ]:
HACKERS_DELIGHT_PROBLEMS = {}
HACKERS_DELIGHT_COMMENTS = {}

for file in HACKERS_DELIGHT_DIRECTORY.iterdir():
    print(file.name)
    with open(file, "r") as f:
        contents = file.read_text()
        HACKERS_DELIGHT_PROBLEMS[file.name] = sexp.loads(contents)
        HACKERS_DELIGHT_COMMENTS[file.name] = [
            line for line in contents.split("\n") if is_comment(line)
        ]
